In [40]:
import numpy as np
import os
from pathlib import Path
import shutil
import json

basedir = Path('/data/ds-siemenspilots/') #sourcedata/mri'

subject = 'alina'

In [26]:
orders = [[0, 2, 4], [4, 2, 0], [2, 0, 4]]

In [66]:
template_json = {"MagneticFieldStrength": 3,
                "ParallelReductionFactorInPlane": 1,
                "RepetitionTime": 0.0,
                "TaskName": "Numerosity estimation",
                "PhaseEncodingDirection": "i",
                'TotalReadoutTime': 0.04,}

In [67]:
from nilearn import image

In [68]:
import json

for session in [1,2,3]:

    target_dir = basedir / f'sub-{subject}' / f'ses-{session}'
    source_dir = basedir / 'sourcedata' / 'mri' / f'sub-{subject}' / f'01_{session:02d}'

    os.makedirs(target_dir, exist_ok=True)

    order = orders[session-1]

    for sd in ['anat', 'fmap', 'func']:
        os.makedirs(target_dir / sd, exist_ok=True)

    # Find anatomical data:
    anat_files = list(source_dir.glob('t1_mprage*.nii.gz'))

    for i, anat_fn in enumerate(anat_files):
        if len(anat_files) == 1:
            target_fn = target_dir / 'anat' / f'sub-{subject}_ses-{session}_T1w.nii.gz'
        else:
            target_fn = target_dir / 'anat' / f'sub-{subject}_ses-{session}_run-{i+1}_T1w.nii.gz'

        shutil.copy(anat_fn, target_fn)

    # Find functional data:

    for i, mb in enumerate(order):

        for repetition in [1, 2]:

            lr = {1: 'LR', 2: 'RL'}[repetition]
            run = i+1 + 3*(repetition-1)

            source_fn = source_dir / f'fmri_g0_sms{mb}_f8p8_r{repetition}.nii.gz'
            target_fn = target_dir / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb}_dir-{lr}_run-{run:02d}_bold.nii.gz'


            shutil.copy(source_fn, target_fn)
            
            json_dict = template_json.copy()
            json_dict['RepetitionTime'] = float(image.load_img(source_fn).header.get_zooms()[3])
            json_dict['PhaseEncodingDirection'] = 'i-' if lr == 'RL' else 'i'

            with open(target_fn.with_suffix('').with_suffix('.json'), 'w') as f:
                json.dump(json_dict, f)

            # If mulitband 1, write it down as sbref

            if mb == 1:
                for run_ in range((repetition-1)*3+1, repetition*3+1):
                    # source_fn = source_dir / f'fmri_g0_sms{mb}_f8p8_r{repetition}_sbref.nii.gz'
                    mbref = image.load_img(source_fn).slicer[0]
                    target_fn = target_dir / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb}_dir-{lr}_run-{run_:02d}_sbref.nii.gz'
                    mbref.to_filename(target_fn)



    for i, mb in enumerate(order):

        for repetition in [1, 2]:
            other_repetition = 3 - repetition + 1
            source_fn = source_dir / f'fmri_g0_sms{mb}_f8p8_r{repetition}.nii.gz'

            if repetition == 1:
                # Get last 5 slices
                topup = image.load_img(source_fn).slicer[:, :, :, -5:]
                bold_fn = Path(f'ses-{session}') / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb}_dir-RL_run-{i+1+3:02d}_bold.nii.gz'
            else:
                # Get first 5 slices
                topup = image.load_img(source_fn).slicer[:, :, :, :5]
                bold_fn = Path(f'ses-{session}') / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb}_dir-LR_run-{i+1:02d}_bold.nii.gz'

            target_fn = target_dir / 'fmap' / f'sub-{subject}_ses-{session}_acq-mb{mb}_dir-{["RL", "LR"][repetition-1]}_epi.nii.gz'
            topup.to_filename(target_fn)

            target_json_fn = target_fn.with_suffix('').with_suffix('.json')

            json_dict['IntendedFor'] = [str(bold_fn)]

            with open(target_json_fn, 'w') as f:
                json.dump(json_dict, f)